In [ ]:
import os
os.environ['CUDA_VIBIBLE_DEVICES']='0'
from ultralytics import YOLO

import torch
import time
from thop import profile
from thop import clever_format
# model_dict = {
#     'v8':  ['n', 's', 'm', 'l', 'x'],
#     'v9':  ['t', 's', 'm', 'c', 'e'],
#     'v10': ['n', 's', 'm', 'b', 'l', 'x'],
#     'v11': ['n', 's', 'm', 'l', 'x'],
#     'v12': ['n', 's', 'm', 'l', 'x'],
#     'v13': ['n', 's', 'l', 'x'],
# }
batch = 16
model_dict = {
    'v8':  ['l', 'x'],
    'v9':  ['c', 'e'],
    'v10': ['l', 'x'],
    'v11': ['l', 'x'],
    'v12': ['l', 'x'],
    'v13': ['l', 'x'],
}
device = "cuda" if torch.cuda.is_available() else "cpu"
for k in model_dict.keys():
    for s in model_dict[k]:
        print(f'yolo{k}{s}')
        det_time_records = []
        model_path = f'/nfs/P111yhchen/code/detection/det_branch/{k}/yolo{k}{s}/weights/best.pt'
        if not os.path.exists(model_path):
            continue
        yolo_model = YOLO(model_path)
        model_det = yolo_model.model.to(device)
        model_det.eval()

        with torch.no_grad():
            input_det = torch.randn(1, 3, 640, 640).to(device)
            macs_det, params_det = profile(model_det, inputs=(input_det, ))
            print(f"detect macs: {macs_det/1e9:.3f}, detect params: {params_det/1e6:.3f}")

            for i in range(200):
                input_det = torch.randn(batch, 3, 640, 640).to(device)

                start_det = time.perf_counter()
                res = model_det(input_det)
                det_time_records.append(1000*(time.perf_counter()-start_det))
            det_time_records.sort()
            mean_det_time = sum(det_time_records[50:-50])/len(det_time_records[50:-50])
            print(f"detect inference: {mean_det_time/batch:.3f} ms")

In [1]:
import os
os.environ['CUDA_VIBIBLE_DEVICES']='0'
from ultralytics import YOLO

# model_dict = {
#     'v8':  ['n', 's', 'm', 'l', 'x'],
#     'v9':  ['t', 's', 'm', 'c', 'e'],
#     'v10': ['n', 's', 'm', 'b', 'l', 'x'],
#     'v11': ['n', 's', 'm', 'l', 'x'],
#     'v12': ['n', 's', 'm', 'l', 'x'],
#     'v13': ['n', 's', 'l', 'x'],
# }
model_dict = {
    # 'v8':  ['n', 's', 'm'],
    'v9':  ['c'],
    # 'v10': ['n', 's', 'm'],
    'v11': ['n', 's', 'm', 'l', 'x'],
    # 'v12': ['n', 's', 'm'],
    # 'v13': ['n', 's'],
}

for k in model_dict.keys():
    for s in model_dict[k]:
        model_path = f'/nfs/P111yhchen/code/detection/det_branch/{k}/yolo{k}{s}/weights/best.pt'
        if not os.path.exists(model_path):
            continue
        model = YOLO(model_path)
        metrics = model.val(
            data = 'ultralytics/cfg/datasets/polyp_2.yaml',
            single_cls=False,
            batch=32,
            # conf=0.001,
            # save_txt=True
        )
        confusion_matrix = metrics.confusion_matrix.matrix
        print(confusion_matrix)
        acc = []
        for ci in range(len(confusion_matrix)-1):
            acc.append(confusion_matrix[ci, ci]/confusion_matrix[:,ci].sum())
        print(acc)

/nfs/P111yhchen/anaconda3/envs/yolov13/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FlashAttention is not available on this device. Using scaled_dot_product_attention instead.
Ultralytics 8.3.63 🚀 Python-3.11.14 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100 80GB PCIe, 81085MiB)
YOLOv9c summary (fused): 384 layers, 25,320,790 parameters, 0 gradients, 102.3 GFLOPs


val: Scanning /local_data/dataset/polyp/detection/patients_complete/labels/val.cache... 679 images, 25 backgrounds, 0 corrupt: 100%|██████████| 704/704 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50      mAP75  mAP50-95): 100%|██████████| 22/22 [00:05<00:00,  4.10it/s]


                   all        704        734      0.671      0.628      0.668      0.495      0.433
          hyperplastic        294        323      0.658      0.596      0.652      0.512      0.428
               adenoma        393        411      0.683      0.659      0.684      0.479      0.438
Speed: 0.6ms preprocess, 3.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/P111yhchen/detection/yolov13/runs/detect/val
[[        152         197        2418]
 [        161         205        1924]
 [         10           9           0]]
[0.47058823529411764, 0.49878345498783455]
Ultralytics 8.3.63 🚀 Python-3.11.14 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100 80GB PCIe, 81085MiB)
YOLOv11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /local_data/dataset/polyp/detection/patients_complete/labels/val.cache... 679 images, 25 backgrounds, 0 corrupt: 100%|██████████| 704/704 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50      mAP75  mAP50-95): 100%|██████████| 22/22 [00:04<00:00,  4.91it/s]


...

                   all        704        734       0.62      0.663      0.657      0.492      0.435
          hyperplastic        294        323       0.59      0.579      0.616      0.497      0.416
               adenoma        393        411       0.65      0.746      0.698      0.487      0.454
Speed: 0.7ms preprocess, 2.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/P111yhchen/detection/yolov13/runs/detect/val4
[[        154         146         933]
 [        149         249         931]
 [         20          16           0]]
[0.47678018575851394, 0.6058394160583942]
Ultralytics 8.3.63 🚀 Python-3.11.14 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100 80GB PCIe, 81085MiB)
YOLOv11l summary (fused): 464 layers, 25,280,854 parameters, 0 gradients, 86.6 GFLOPs


val: Scanning /local_data/dataset/polyp/detection/patients_complete/labels/val.cache... 679 images, 25 backgrounds, 0 corrupt: 100%|██████████| 704/704 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50      mAP75  mAP50-95): 100%|██████████| 22/22 [00:05<00:00,  4.03it/s]


                   all        704        734       0.66      0.627      0.671      0.507      0.443
          hyperplastic        294        323      0.625      0.622      0.658      0.519      0.441
               adenoma        393        411      0.695      0.631      0.685      0.494      0.445
Speed: 0.6ms preprocess, 3.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /home/P111yhchen/detection/yolov13/runs/detect/val5
[[        158         196        3582]
 [        157         210        2884]
 [          8           5           0]]
[0.4891640866873065, 0.5109489051094891]
Ultralytics 8.3.63 🚀 Python-3.11.14 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100 80GB PCIe, 81085MiB)
YOLOv11x summary (fused): 464 layers, 56,829,334 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /local_data/dataset/polyp/detection/patients_complete/labels/val.cache... 679 images, 25 backgrounds, 0 corrupt: 100%|██████████| 704/704 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50      mAP75  mAP50-95): 100%|██████████| 22/22 [00:06<00:00,  3.40it/s]


                   all        704        734      0.609      0.655      0.668      0.511       0.45
          hyperplastic        294        323       0.57      0.588      0.622      0.496      0.419
               adenoma        393        411      0.647      0.723      0.714      0.526      0.481
Speed: 0.6ms preprocess, 5.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /home/P111yhchen/detection/yolov13/runs/detect/val6
[[        173         133         847]
 [        131         265         783]
 [         19          13           0]]
[0.5356037151702786, 0.6447688564476886]
